In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'name': ['ray', 'jack', 'lucy', 'bob', 'candy', 'luck', 'body', 'rain', 'perk', 'dd', 'ppd'],
                   'sex': ['nv', 'nan', 'nv', 'nan', 'nv', 'nan', 'nv', 'nan', 'nv', 'nan', 'nan'],
                   'age': [10, 30, 20, 15, 50, 15, 30, 20, 20, 30, 15],
                   'height': [180, 168, 170, 190, 180, 178, 188, 180, 170, 160, 159],
                   'city': ['hangzhou', 'beijing', 'hangzhou', 'chengdu', 'beijing', 'suzhou', 'hangzhou', 'beijing', 'hangzhou', 'chengdu', 'suzhou']},
                  columns=['name', 'sex', 'age', 'city', 'height'])
print(df)

     name  sex  age      city  height
0     ray   nv   10  hangzhou     180
1    jack  nan   30   beijing     168
2    lucy   nv   20  hangzhou     170
3     bob  nan   15   chengdu     190
4   candy   nv   50   beijing     180
5    luck  nan   15    suzhou     178
6    body   nv   30  hangzhou     188
7    rain  nan   20   beijing     180
8    perk   nv   20  hangzhou     170
9      dd  nan   30   chengdu     160
10    ppd  nan   15    suzhou     159


In [2]:
# 分组计算
group1 = df.groupby('sex')
sex_age_mean = group1[['age']].mean()
print(sex_age_mean)
print('----------------')
group2 = df.groupby(['sex', 'city'])
sexCity_age_mean = group2[['age', 'height']].mean()
sexCity_age_mean1 = group2[['age']].agg([np.mean, np.median, np.sum])
sexCity_age_mean2 = group1.agg({'age': [np.mean, np.sum], 'height': np.mean})
print(sexCity_age_mean)
print('------------------')
print(sexCity_age_mean1)
print('------------------')
print(sexCity_age_mean2)

           age
sex           
nan  20.833333
nv   26.000000
----------------
               age  height
sex city                  
nan beijing   25.0   174.0
    chengdu   22.5   175.0
    suzhou    15.0   168.5
nv  beijing   50.0   180.0
    hangzhou  20.0   177.0
------------------
               age           
              mean median sum
sex city                     
nan beijing   25.0   25.0  50
    chengdu   22.5   22.5  45
    suzhou    15.0   15.0  30
nv  beijing   50.0   50.0  50
    hangzhou  20.0   20.0  80
------------------
           age      height
          mean  sum   mean
sex                       
nan  20.833333  125  172.5
nv   26.000000  130  177.6


In [3]:
zscore = lambda x: (x - x.mean()) / x.std()
a = df.groupby('sex').transform(zscore)
print(a)

         age    height
0  -1.055009  0.312984
1   1.245505 -0.367057
2  -0.395628 -0.991117
3  -0.792594  1.427442
4   1.582513  0.312984
5  -0.792594  0.448625
6   0.263752  1.356265
7  -0.113228  0.611761
8  -0.395628 -0.991117
9   1.245505 -1.019602
10 -0.792594 -1.101170


In [4]:
filter1 = lambda x: x['age'].mean() <= 20
group2 = df.groupby(['sex', 'city'])
print(group2.filter(filter1))

    name  sex  age      city  height
0    ray   nv   10  hangzhou     180
2   lucy   nv   20  hangzhou     170
5   luck  nan   15    suzhou     178
6   body   nv   30  hangzhou     188
8   perk   nv   20  hangzhou     170
10   ppd  nan   15    suzhou     159


In [5]:
level1 = df.set_index(['sex', 'city'])
level_group = level1.groupby(level=[0, 1])
print(level1)
print(level1.sort_index())
print(level_group.mean())

               name  age  height
sex city                        
nv  hangzhou    ray   10     180
nan beijing    jack   30     168
nv  hangzhou   lucy   20     170
nan chengdu     bob   15     190
nv  beijing   candy   50     180
nan suzhou     luck   15     178
nv  hangzhou   body   30     188
nan beijing    rain   20     180
nv  hangzhou   perk   20     170
nan chengdu      dd   30     160
    suzhou      ppd   15     159
               name  age  height
sex city                        
nan beijing    jack   30     168
    beijing    rain   20     180
    chengdu     bob   15     190
    chengdu      dd   30     160
    suzhou     luck   15     178
    suzhou      ppd   15     159
nv  beijing   candy   50     180
    hangzhou    ray   10     180
    hangzhou   lucy   20     170
    hangzhou   body   30     188
    hangzhou   perk   20     170
               age  height
sex city                  
nan beijing   25.0   174.0
    chengdu   22.5   175.0
    suzhou    15.0   168.5
nv  bei

In [6]:
# 组的一些属性
print(group2.size())

sex  city    
nan  beijing     2
     chengdu     2
     suzhou      2
nv   beijing     1
     hangzhou    4
dtype: int64


In [7]:
print(group2.groups)        # 得到每个索引组在源数据中的索引位置

{('nan', 'beijing'): Int64Index([1, 7], dtype='int64'), ('nan', 'chengdu'): Int64Index([3, 9], dtype='int64'), ('nan', 'suzhou'): Int64Index([5, 10], dtype='int64'), ('nv', 'beijing'): Int64Index([4], dtype='int64'), ('nv', 'hangzhou'): Int64Index([0, 2, 6, 8], dtype='int64')}


In [8]:
print(group2.get_group(('nan', 'beijing')))     # 得到包含索引组的所有数据

   name  sex  age     city  height
1  jack  nan   30  beijing     168
7  rain  nan   20  beijing     180


In [9]:
for s, c in group2:
    print(s)
    print(c.shape)

('nan', 'beijing')
(2, 5)
('nan', 'chengdu')
(2, 5)
('nan', 'suzhou')
(2, 5)
('nv', 'beijing')
(1, 5)
('nv', 'hangzhou')
(4, 5)


In [10]:
# 数据透视表
print(df.pivot_table(index='sex', columns='city', values='height', aggfunc=[np.mean, len]))  # 还有margins增加总计行列
print('--------------------------------------------------------------')
pivot1 = df.pivot_table(index=['sex', 'age'], columns='city', values='height', aggfunc=[np.mean, len], fill_value=0)
print(pivot1)

        mean                             len                        
city beijing chengdu hangzhou suzhou beijing chengdu hangzhou suzhou
sex                                                                 
nan    174.0   175.0      NaN  168.5     2.0     2.0      NaN    2.0
nv     180.0     NaN    177.0    NaN     1.0     NaN      4.0    NaN
--------------------------------------------------------------
           mean                             len                        
city    beijing chengdu hangzhou suzhou beijing chengdu hangzhou suzhou
sex age                                                                
nan 15        0     190        0  168.5       0       1        0      2
    20      180       0        0    0.0       1       0        0      0
    30      168     160        0    0.0       1       1        0      0
nv  10        0       0      180    0.0       0       0        1      0
    20        0       0      170    0.0       0       0        2      0
    30        0 

In [11]:
# 重塑层次化索引
# stack()  将数据最内层的列旋转到行上    pivot1.stack()
# unstack() 将数据最内层的行旋转到列上   pivot1.unstack()

In [12]:
# 数据交叉表：用于计算分组频率
cross1 = pd.crosstab(index=df['sex'], columns=df['age'])
print(cross1)

age  10  15  20  30  50
sex                    
nan   0   3   1   2   0
nv    1   0   2   1   1


In [13]:
# 数据统计
print(df.sample(n=5))
w = [0.2, 0.3, 0.5]
print(df.head(3).sample(n=2, weights=w, replace=False))     # replace默认不放回

   name  sex  age      city  height
3   bob  nan   15   chengdu     190
5  luck  nan   15    suzhou     178
1  jack  nan   30   beijing     168
9    dd  nan   30   chengdu     160
6  body   nv   30  hangzhou     188
   name  sex  age      city  height
2  lucy   nv   20  hangzhou     170
1  jack  nan   30   beijing     168


In [14]:
print(df.describe().round(1))

        age  height
count  11.0    11.0
mean   23.2   174.8
std    11.2    10.3
min    10.0   159.0
25%    15.0   169.0
50%    20.0   178.0
75%    30.0   180.0
max    50.0   190.0


In [15]:
print(df['age'].max())
print(df['age'].idxmax())       # 返回最大值的索引，也有df['age'].idxmin()
print(df.loc[4, 'age'])

50
4
50


In [16]:
# 协方差
print(df.cov())
print(df['age'].cov(df['height']))      # 两变量之间的协方差

               age      height
age     126.363636    0.636364
height    0.636364  105.763636
0.6363636363636379


In [17]:
# 相关系数
print(df.corr())
print(df['age'].corr(df['height']))     # 两变量之间的相关系数

             age    height
age     1.000000  0.005505
height  0.005505  1.000000
0.005504604364704426
